In [1]:
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import LSTM
from keras import callbacks
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [2]:
df_train = pd.read_csv(
    'train.csv', usecols=[1, 2, 3, 4, 5],
    dtype={'onpromotion': bool},
    converters={'unit_sales': lambda u: np.log1p(
        float(u)) if float(u) > 0 else 0},
    parse_dates=["date"],
    skiprows=range(1, 66458909)  # 2016-01-01
)

In [3]:
df_test = pd.read_csv(
    "test.csv", usecols=[0, 1, 2, 3, 4],
    dtype={'onpromotion': bool},
    parse_dates=["date"]  # , date_parser=parser
).set_index(
    ['store_nbr', 'item_nbr', 'date']
)

In [4]:
items = pd.read_csv(
    "items.csv",
).set_index("item_nbr")

In [5]:
df_2017 = df_train.loc[df_train.date>=pd.datetime(2017,1,1)]
del df_train


In [6]:
promo_2017_train = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["onpromotion"]].unstack(
        level=-1).fillna(False)
promo_2017_train.columns = promo_2017_train.columns.get_level_values(1)
promo_2017_test = df_test[["onpromotion"]].unstack(level=-1).fillna(False)
promo_2017_test.columns = promo_2017_test.columns.get_level_values(1)
promo_2017_test = promo_2017_test.reindex(promo_2017_train.index).fillna(False)
promo_2017 = pd.concat([promo_2017_train, promo_2017_test], axis=1)
del promo_2017_test, promo_2017_train

In [7]:
df_2017 = df_2017.set_index(
    ["store_nbr", "item_nbr", "date"])[["unit_sales"]].unstack(
        level=-1).fillna(0)
df_2017.columns = df_2017.columns.get_level_values(1)

In [8]:
items = items.reindex(df_2017.index.get_level_values(1))

def get_timespan(df, dt, minus, periods, freq='D'):
    return df[pd.date_range(dt - timedelta(days=minus), periods=periods, freq=freq)]

In [9]:
def prepare_dataset(t2017, is_train=True):
    X = pd.DataFrame({
        "day_1_2017": get_timespan(df_2017, t2017, 1, 1).values.ravel(),
        "mean_3_2017": get_timespan(df_2017, t2017, 3, 3).mean(axis=1).values,
        "mean_7_2017": get_timespan(df_2017, t2017, 7, 7).mean(axis=1).values,
        "mean_14_2017": get_timespan(df_2017, t2017, 14, 14).mean(axis=1).values,
        "mean_30_2017": get_timespan(df_2017, t2017, 30, 30).mean(axis=1).values,
        "mean_60_2017": get_timespan(df_2017, t2017, 60, 60).mean(axis=1).values,
        "mean_140_2017": get_timespan(df_2017, t2017, 140, 140).mean(axis=1).values,
        "promo_14_2017": get_timespan(promo_2017, t2017, 14, 14).sum(axis=1).values,
        "promo_60_2017": get_timespan(promo_2017, t2017, 60, 60).sum(axis=1).values,
        "promo_140_2017": get_timespan(promo_2017, t2017, 140, 140).sum(axis=1).values
    })
    for i in range(7):
        X['mean_4_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 28-i, 4, freq='7D').mean(axis=1).values
        X['mean_20_dow{}_2017'.format(i)] = get_timespan(df_2017, t2017, 140-i, 20, freq='7D').mean(axis=1).values
    for i in range(16):
        X["promo_{}".format(i)] = promo_2017[
            t2017 + timedelta(days=i)].values.astype(np.uint8)
    if is_train:
        y = df_2017[
            pd.date_range(t2017, periods=16)
        ].values
        return X, y
    return X


In [10]:
print("Preparing dataset...")
t2017 = date(2017, 5, 31)
X_l, y_l = [], []
for i in range(6):
    delta = timedelta(days=7 * i)
    X_tmp, y_tmp = prepare_dataset(
        t2017 + delta
    )
    X_l.append(X_tmp)
    y_l.append(y_tmp)

Preparing dataset...


In [11]:
X_train = pd.concat(X_l, axis=0)
y_train = np.concatenate(y_l, axis=0)
del X_l, y_l
X_val, y_val = prepare_dataset(date(2017, 7, 26))
X_test = prepare_dataset(date(2017, 8, 16), is_train=False)


In [12]:
stores_items = pd.DataFrame(index=df_2017.index)
test_ids = df_test[['id']]

items = items.reindex( stores_items.index.get_level_values(1) )

In [13]:
X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
X_val = X_val.as_matrix()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

In [14]:
model = Sequential()
model.add(LSTM(32, input_shape=(X_train.shape[1],X_train.shape[2])))
model.add(Dropout(.1))
model.add(Dense(32))
model.add(Dropout(.2))
model.add(Dense(1))
model.compile(loss = 'mse', optimizer='adam', metrics=['mse'])

In [15]:
N_EPOCHS = 5

val_pred = []
test_pred = []
# wtpath = 'weights.hdf5'  # To save best epoch. But need Keras bug to be fixed first.
sample_weights=np.array( pd.concat([items["perishable"]] * 6) * 0.25 + 1 )

In [16]:
for i in range(16):
    print("=" * 50)
    print("Step %d" % (i+1))
    print("=" * 50)
    y = y_train[:, i]
    xv = X_val
    yv = y_val[:, i]
    model.fit(X_train, y, batch_size = 512, epochs = N_EPOCHS, verbose=2,
               sample_weight=sample_weights, validation_data=(xv,yv) ) 
    val_pred.append(model.predict(X_val))
    test_pred.append(model.predict(X_test))

Step 1
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 26s - loss: 0.3990 - mean_squared_error: 0.3760 - val_loss: 0.3116 - val_mean_squared_error: 0.3116
Epoch 2/5
 - 24s - loss: 0.3554 - mean_squared_error: 0.3363 - val_loss: 0.3021 - val_mean_squared_error: 0.3021
Epoch 3/5
 - 23s - loss: 0.3457 - mean_squared_error: 0.3275 - val_loss: 0.2972 - val_mean_squared_error: 0.2972
Epoch 4/5
 - 22s - loss: 0.3406 - mean_squared_error: 0.3228 - val_loss: 0.2982 - val_mean_squared_error: 0.2982
Epoch 5/5
 - 23s - loss: 0.3384 - mean_squared_error: 0.3209 - val_loss: 0.2987 - val_mean_squared_error: 0.2987
Step 2
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 25s - loss: 0.3660 - mean_squared_error: 0.3452 - val_loss: 0.3426 - val_mean_squared_error: 0.3426
Epoch 2/5
 - 24s - loss: 0.3619 - mean_squared_error: 0.3415 - val_loss: 0.3266 - val_mean_squared_error: 0.3266
Epoch 3/5
 - 23s - loss: 0.3615 - mean_squared_error: 0.3411 - val_loss: 0.3282 - val_mea

Epoch 2/5
 - 13s - loss: 0.4239 - mean_squared_error: 0.4017 - val_loss: 0.3888 - val_mean_squared_error: 0.3888
Epoch 3/5
 - 17s - loss: 0.4219 - mean_squared_error: 0.3999 - val_loss: 0.3895 - val_mean_squared_error: 0.3895
Epoch 4/5
 - 16s - loss: 0.4202 - mean_squared_error: 0.3983 - val_loss: 0.3899 - val_mean_squared_error: 0.3899
Epoch 5/5
 - 17s - loss: 0.4200 - mean_squared_error: 0.3980 - val_loss: 0.3870 - val_mean_squared_error: 0.3870
Step 13
Train on 1005090 samples, validate on 167515 samples
Epoch 1/5
 - 17s - loss: 0.4149 - mean_squared_error: 0.3916 - val_loss: 0.3798 - val_mean_squared_error: 0.3798
Epoch 2/5
 - 15s - loss: 0.4067 - mean_squared_error: 0.3840 - val_loss: 0.3788 - val_mean_squared_error: 0.3788
Epoch 3/5
 - 15s - loss: 0.4055 - mean_squared_error: 0.3829 - val_loss: 0.3806 - val_mean_squared_error: 0.3806
Epoch 4/5
 - 16s - loss: 0.4046 - mean_squared_error: 0.3821 - val_loss: 0.3769 - val_mean_squared_error: 0.3769
Epoch 5/5
 - 16s - loss: 0.4039 - m

In [17]:
n_public = 5 # Number of days in public test set
weights=pd.concat([items["perishable"]]) * 0.25 + 1
print("Unweighted validation mse: ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose()) )
print("Full validation mse:       ", mean_squared_error(
    y_val, np.array(val_pred).squeeze(axis=2).transpose(), sample_weight=weights) )
print("'Public' validation mse:   ", mean_squared_error(
    y_val[:,:n_public], np.array(val_pred).squeeze(axis=2).transpose()[:,:n_public], 
    sample_weight=weights) )
print("'Private' validation mse:  ", mean_squared_error(
    y_val[:,n_public:], np.array(val_pred).squeeze(axis=2).transpose()[:,n_public:], 
    sample_weight=weights) )

Unweighted validation mse:  0.369163668494
Full validation mse:        0.368709261587
'Public' validation mse:    0.339996983697
'Private' validation mse:   0.381760296992


In [18]:
y_test = np.array(test_pred).squeeze(axis=2).transpose()
df_preds = pd.DataFrame(
    y_test, index=stores_items.index,
    columns=pd.date_range("2017-08-16", periods=16)
).stack().to_frame("unit_sales")
df_preds.index.set_names(["store_nbr", "item_nbr", "date"], inplace=True)

In [19]:
submission = test_ids.join(df_preds, how="left").fillna(0)
submission["unit_sales"] = np.clip(np.expm1(submission["unit_sales"]), 0, 1000)
submission.to_csv('lstm.csv', float_format='%.4f', index=None)